In [1]:
import numpy as np
import pandas as pd
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from fuzzywuzzy import fuzz
import re
import nltk
import logging
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras import layers
from nltk.tokenize import word_tokenize
from numpy.linalg import norm
nltk.download('stopwords')
from gensim.parsing.preprocessing import remove_stopwords
import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
from keras.layers import Embedding

Using TensorFlow backend.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dhwanikapadia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data=pd.read_csv('outfit_combinations.csv')
df=pd.read_csv('Full+data.csv')

In [3]:
df.shape

(42373, 256)

In [4]:
df.head()

,product_id,brand,mpn,name,description,brand_category,created_at,updated_at,deleted_at,brand_canonical_url,...,Unnamed: 246,Unnamed: 247,Unnamed: 248,Unnamed: 249,Unnamed: 250,Unnamed: 251,Unnamed: 252,Unnamed: 253,Unnamed: 254,Unnamed: 255
0,01DSRPSZTDW2PGK1YWYXJGKZZ0,FILA,400010319073,Original Fitness Sneakers,Vintage Fitness leather sneakers with logo pri...,TheMensStore/Shoes/Sneakers/LowTop,2019-11-15 23:36:38.98161+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/fila-original-...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01DSQXJBX0R7DCW7KTAC1SW547,CHANEL,400011497371,HAT,NaN,Unknown,2019-11-15 16:15:34.809725+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/chanel-hat/pro...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01DPGV8TGRAB993PF7Z3YWG2VR,Frame,LWBT0099,Petit Oval Buckle Belt,A Timeless Leather Belt Crafted From Smooth Co...,Accessories,2019-10-06 15:33:38.451504+00,2019-12-19 20:40:30.786144+00,NaN,https://frame-store.com/products/petit-oval-bu...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01DSR8G3F7DBRTMP8THF97XSQ2,Lilly Pulitzer Kids,400011844797,Little Gir's & Girl's Ariana One-Piece UPF 50+...,Pretty ruffle sleeves and trim elevate essenti...,"JustKids/Girls214/Girls/SwimwearCoverups,JustK...",2019-11-15 19:26:34.947426+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/lilly-pulitzer...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01DSR8G5GP519DEDCSKBMWQVK5,Kissy Kissy,400094474585,Baby Girl's Endearing Elephants Pima Cotton Co...,Versatile convertible gown with elephant applique,JustKids/Baby024months/InfantGirls/FootiesRompers,2019-11-15 19:26:36.995998+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/kissy-kissy-ba...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df=df.dropna(axis=1,how='all')
data=data.merge(df[['product_id','description','details', 'brand_category']],on='product_id',how='inner')

### Pre-processing

In [6]:
def clean_text(x):
        try:
            x = re.sub(r'<.*?>', '',x)
            x = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", x)
            x = re.sub(r"\'s", " \'s", x)
            x = re.sub(r"\'ve", " \'ve", x)
            x = re.sub(r"n\'t", " n\'t", x)
            x = re.sub(r"\'re", " \'re", x)
            x = re.sub(r"\'d", " \'d", x)
            x = re.sub(r"\'ll", " \'ll", x)
            x = re.sub(r",", " , ", x)
            x = re.sub(r"!", " ! ", x)
            x = re.sub(r"\(", "", x)
            x = re.sub(r"\)", "", x)
            x = re.sub(r"\?", "", x)
            x = re.sub(r"/", "", x)
            x = re.sub(r"\s{2,}", " ", x)
            return x.lower()
        except:
            return ''
        
def lemmatizer(x):
        return ' '.join([token.lemma_ for token in nlp(x)])

In [7]:
columns=['outfit_id', 'product_id','outfit_item_type','brand','product_full_name','description', 'details', 'brand_category']
data = data[columns]


### Example A: Function

In [8]:
def get_recommendation_product_id(data, product_id):
    df = data[data['outfit_item_type'] == product_id[0]]
    product_id_match = pd.DataFrame({'product_id': df['product_id'].unique()})
    product_id_match['match'] = product_id_match['product_id'].apply(lambda x: fuzz.token_set_ratio(product_id[-1], x))
    product_id_match = product_id_match.sort_values(by = 'match', ascending = False)
    product_id = product_id_match.iloc[0,0]
    outfits = data[data['product_id'] == product_id][['outfit_id', 'product_id']].drop_duplicates()['outfit_id'].tolist()
    return data[data['outfit_id'].apply(lambda x: True if x in outfits else False)][['outfit_id', 'product_id', 'product_full_name', 'outfit_item_type']].reset_index(drop = True).sort_values(by = 'outfit_id'), product_id_match.iloc[0,1] 

In [9]:
data['text'] = ''
for key in ['outfit_item_type', 'brand', 'product_full_name', 'description', 'details', 'brand_category']:
    data['text'] = data['text'] + ' ' + data[key]

In [10]:
%%time
data['text'] = data['text'].apply(clean_text)
data['text'] = data['text'].apply(remove_stopwords)
data['text'] = data['text'].apply(lemmatizer)

CPU times: user 11.8 s, sys: 111 ms, total: 11.9 s
Wall time: 11.9 s


### Example B: Function

In [15]:
def get_recommendation_desc(data, query):
    cat = query[0]
    df = data[data['outfit_item_type'] == cat]
    
    tokenizer = Tokenizer(num_words=500, oov_token="UNKNOWN_TOKEN")
    tokenizer.fit_on_texts(df['text'])

    X = tokenizer.texts_to_sequences(df['text'])
    
    query = clean_text(query[1])
    query = remove_stopwords(query)
    query = lemmatizer(query)

    query=[query]
    
    X = list(df['text'].values)

    vectorizer = TfidfVectorizer(min_df=3, max_df=0.9) 
                              
    X = vectorizer.fit_transform(X)
    query = vectorizer.transform(query)
    tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
    tfidf_query = pd.DataFrame(query.toarray(), columns=vectorizer.get_feature_names())                         
    
    df['similarity'] = cosine_similarity(tfidf_df, tfidf_query)

    product_id = df.sort_values(by = 'similarity', ascending = False).reset_index(drop = True).head(1)['product_id']
    
    return get_recommendation_product_id(data, (cat, product_id))

In [12]:
### Input function: Example A

outfit = input('Enter outfit type: ')   
prod_id = input('Enter product id: ')

query=(outfit,prod_id)

df,score = get_recommendation_product_id(data,query)
df

Enter outfit type: shoe
Enter product id: 01DMBRYVA2ZFDYRYY5TRQZJTBD


,outfit_id,product_id,product_full_name,outfit_item_type
0,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2P5H24WK0HTK4R0A1,Slim Knit Skirt,bottom
2,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2PEPWFTT7RMP5AA1T,Rib Mock Neck Tank,top
4,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2S5T9W793F4CY41HE,medium margaux leather satchel,accessory1
6,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2ZFDYRYY5TRQZJTBD,Penelope Mid Cap Toe Pump,shoe
1,01DMHCX50CFX5YNG99F3Y65GQW,01DMBRYVA2P5H24WK0HTK4R0A1,Slim Knit Skirt,bottom
3,01DMHCX50CFX5YNG99F3Y65GQW,01DMBRYVA2PEPWFTT7RMP5AA1T,Rib Mock Neck Tank,top
7,01DMHCX50CFX5YNG99F3Y65GQW,01DMBRYVA2ZFDYRYY5TRQZJTBD,Penelope Mid Cap Toe Pump,shoe
10,01DMHCX50CFX5YNG99F3Y65GQW,01DMHCNT41E14QWP503V7CT9G6,Crystal Clutch,accessory1
8,01DMHRX35M2DPVYVQ1PNER4S4B,01DMBRYVA2ZFDYRYY5TRQZJTBD,Penelope Mid Cap Toe Pump,shoe
11,01DMHRX35M2DPVYVQ1PNER4S4B,01DMHCNT41E14QWP503V7CT9G6,Crystal Clutch,accessory1


In [16]:
### Input function: Example B

outfit = input('Enter outfit type: ')   
description = input('Enter description: ') 

query=(outfit,description)

df,score = get_recommendation_desc(data,query)
df



Enter outfit type: bottom
Enter description: slim fitting, straight leg pant with a center back zipper and slightly cropped leg


,outfit_id,product_id,product_full_name,outfit_item_type
0,01DVBQN0A1NQVQY805R202YW8H,01DT0DJQQ3J6EAWSPMEYCY5AYB,Heather C-Chain Leopard-Print Calf Hair & Leat...,shoe
3,01DVBQN0A1NQVQY805R202YW8H,01DT0DKEZ2DQZ9SXHE34790QGC,Woven Boyfriend V-Neck Tee,top
5,01DVBQN0A1NQVQY805R202YW8H,01DT513A3WD9WT4HCZAJ4FVTWS,Jenna stretch-cotton twill straight-leg pants,bottom
11,01DVBQN0A1NQVQY805R202YW8H,01DT515RAX3SAPGFWARM3X7993,Croc-effect leather belt bag,accessory1
17,01DVBQN0A1NQVQY805R202YW8H,01DT518N0WCC9SBA7XZ7HYKF6S,Cotton-blend twill trench coat,accessory2
25,01DVBQN0A21NSDFGJC5VZVS7S2,01DT37D24EZZY0AZ61W5VY8WNR,Bente Zebra-Print Calf Hair Mules,shoe
4,01DVBQN0A21NSDFGJC5VZVS7S2,01DT0DKEZ2DQZ9SXHE34790QGC,Woven Boyfriend V-Neck Tee,top
6,01DVBQN0A21NSDFGJC5VZVS7S2,01DT513A3WD9WT4HCZAJ4FVTWS,Jenna stretch-cotton twill straight-leg pants,bottom
18,01DVBQN0A21NSDFGJC5VZVS7S2,01DT518N0WCC9SBA7XZ7HYKF6S,Cotton-blend twill trench coat,accessory2
12,01DVBQN0A21NSDFGJC5VZVS7S2,01DT515RAX3SAPGFWARM3X7993,Croc-effect leather belt bag,accessory1
